Feb 28

It has turned out that the validation of the current
ower-fb-3 dataset are not useful as all the ground truth
classes are false for the first 300 entities.

The script for building the OWER dataset needs to be
examined and corrected. Estimated, the most frequent
classes should be true at least for every 100th entity.

# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.lib.pretty import pretty
from pathlib import Path

from dao.ryn.ryn_dir import RynDir

# Config

In [ ]:
ryn_dataset_dir = 'data/ryn/irt.cde.cde.1.clean'
# ryn_dataset_dir = 'data/ryn/irt.cde.irt.1.clean'
# ryn_dataset_dir = 'data/ryn/irt.cde.irt.5.clean'
# ryn_dataset_dir = 'data/ryn/irt.cde.irt.15.clean'
# ryn_dataset_dir = 'data/ryn/irt.cde.irt.30.clean'
# ryn_dataset_dir = 'data/ryn/irt.fb.irt.1.clean'
# ryn_dataset_dir = 'data/ryn/irt.fb.irt.5.clean'
# ryn_dataset_dir = 'data/ryn/irt.fb.irt.15.clean'
# ryn_dataset_dir = 'data/ryn/irt.fb.irt.30.clean'
# ryn_dataset_dir = 'data/ryn/irt.fb.owe.1.clean'

# 1. Check Ryn Directory

Check that the Ryn Directory exists and that all required
files exist.

In [ ]:
ryn_dir = RynDir('Ryn Directory', Path(ryn_dataset_dir))
ryn_dir.check()

debug = False
if debug:
    print(ryn_dir)